<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/BGL_20_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


#copy log file
!cp '/content/drive/MyDrive/BGL_20_5/X_train_index' '/content/'
!cp '/content/drive/MyDrive/BGL_20_5/Xabnorm_test_index' '/content/'
!cp '/content/drive/MyDrive/BGL_20_5/Xnorm_test_index' '/content/'
!cp '/content/drive/MyDrive/BGL_20_5/index2embed' '/content/'
!cp '/content/drive/MyDrive/BGL_20_5/log2index' '/content/'

In [ ]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:

with open('/content/log2index') as f:
    templates = [line.strip() for line in f.readlines()]

In [ ]:
templates[:5]

['ras kernel info instruction cache parity error corrected',
 'ras linkcard info midplaneswitchcontroller performing bit sparing on bit',
 'ras kernel info generating',
 'ras kernel info ddr errors s detected and corrected on rank symbol bit',
 'ras kernel info edram error s dcr detected and corrected']

In [ ]:
#Json file name
with open('/content/index2embed') as f:
    embeddings = [json.loads(line) for line in f.readlines()]

len(embeddings[0])

384

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
embed = np.array(embeddings)
matrix = np.zeros((len(templates),len(templates)))
max_similarity = 0

for i in range(len(templates)):
  for j in range(i+1,len(templates)) :

      similarity = cosine_similarity(embed[i].reshape(1, -1),embed[j].reshape(1, -1))
      matrix[i,j] = similarity

      if (similarity > 0.96) :
        print('similarity = ',similarity)
        print(templates[i])
        print(templates[j])
        print('-----------------------------------')

similarity =  [[0.96168234]]
ras kernel info edram error s dcr detected and corrected
ras kernel info edram error s dcr detected and corrected over seconds
-----------------------------------
similarity =  [[0.96626515]]
ras kernel info total of ddr error s detected and corrected
ras kernel info total of ddr error s detected and corrected over seconds
-----------------------------------
similarity =  [[0.99162441]]
ras app fatal ciod error loading invalid or missing program image no such file or directory
ras app fatal ciod error loading make invalid or missing program image no such file or directory
-----------------------------------
similarity =  [[0.96960119]]
ras app fatal ciod error loading invalid or missing program image no such file or directory
ras app fatal ciod error loading runtimetime invalid or missing program image no such file or directory
-----------------------------------
similarity =  [[0.99374737]]
ras app fatal ciod error loading invalid or missing program image 

KeyboardInterrupt: ignored

In [ ]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, sequence_length, dropout_prob): #input_dim= embeddings_dim
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.decoder = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        encoded, _ = self.encoder(x)
        #print(encoded.shape)
        encoded = encoded[:,-1:,:] # output of last cell
        encoded = self.fc(encoded)
        #print(encoded.shape)
        input_decode = torch.tile(encoded, (1, self.sequence_length, 1))
        decoded, _ = self.decoder(input_decode)
        return decoded

In [ ]:
sequence_length = 20
input_size = len(embeddings[0]) #embedding vector dimension
hidden_dim = 128
num_layers = 2
dropout_prob = 0.2

model = LSTMAutoencoder(input_size, hidden_dim, 2, sequence_length, dropout_prob).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(384, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (decoder): LSTM(128, 384, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=128, bias=True)
)

In [ ]:
data = torch.randn(1,sequence_length, input_size) #batch_size=1
model(data.to(device)).shape

torch.Size([1, 20, 384])

In [ ]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 2926720


In [ ]:
window_size = sequence_length
num_sessions = 0
sequences = {}

line = [i for i in range(20)]
print(line)

for i in range(len(line) - window_size):
    seq = tuple(line[i:i + window_size])
    if seq not in sequences:
      sequences[seq] = [ embeddings[i] for i in seq]


print(sequences.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
dict_keys([(0, 1, 2, 3, 4), (1, 2, 3, 4, 5), (2, 3, 4, 5, 6), (3, 4, 5, 6, 7), (4, 5, 6, 7, 8), (5, 6, 7, 8, 9), (6, 7, 8, 9, 10), (7, 8, 9, 10, 11), (8, 9, 10, 11, 12), (9, 10, 11, 12, 13), (10, 11, 12, 13, 14), (11, 12, 13, 14, 15), (12, 13, 14, 15, 16), (13, 14, 15, 16, 17), (14, 15, 16, 17, 18)])


In [ ]:
name = 'X_train_index'
window_size = sequence_length
sequences = {}
#outputs = []
with open('/content/' + name, 'r') as f:
        for row in f:
            #seq = [ int(i) for i in row.strip().split()]
            seq = tuple([ int(i) for i in row.strip().split()])
            if seq not in sequences:
              sequences[seq] = [ embeddings[i] for i in seq]

print('the length of dictionary : ', len(sequences))

dataset = TensorDataset(torch.tensor(list(sequences.values()), dtype=torch.float))

the length of dictionary :  68942


In [ ]:
list(sequences.keys())[:10]

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0),
 (0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0),
 (2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0)]

In [ ]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
print(len(dataloader)) # regarding to batch size


for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

270
torch.Size([256, 20, 384])


In [ ]:
def adjust_learning_rate(optimizer, epoch, lr_step=(30,50,60,80), lr_decay_ratio=0.5):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [ ]:
num_epochs = 100
learning_rate = 0.001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [ ]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in tqdm(range(num_epochs), desc="Processing Rows"):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, (train_loss*100) / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Processing Rows:   1%|          | 1/100 [00:11<18:33, 11.25s/it]

Epoch [1/100], train_loss: 0.1342


Processing Rows:   2%|▏         | 2/100 [00:21<17:34, 10.76s/it]

Epoch [2/100], train_loss: 0.0808


Processing Rows:   3%|▎         | 3/100 [00:32<17:18, 10.71s/it]

...

Epoch [64/100], train_loss: 0.0056


Processing Rows:  65%|██████▌   | 65/100 [11:51<06:23, 10.96s/it]

Epoch [65/100], train_loss: 0.0056


Processing Rows:  65%|██████▌   | 65/100 [11:55<06:25, 11.01s/it]


KeyboardInterrupt: ignored

In [ ]:
# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/BGL1_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [ ]:
len(embeddings)

736

In [ ]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [ ]:
test_normal_loader, test_normal_length = generate('Xnorm_test_index')
test_abnormal_loader, test_abnormal_length = generate('Xabnorm_test_index')

Number of sessions(Xnorm_test_index): 64372
Number of sessions(Xabnorm_test_index): 12691


In [ ]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in tqdm(test_normal_loader.keys(), desc="Processing Rows"):
          seq = [embeddings[temp] for temp in line]
          seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
          output = model(seq)

          loss = criterion(output, seq)
          #print(loss.cpu().detach().numpy())
          if (loss.cpu().detach().numpy()>threshold):
            FP += test_normal_loader[line] # numbers of that set we have

  with torch.no_grad():
      for line in tqdm(test_abnormal_loader.keys(), desc="Processing Rows"):
          seq = [embeddings[temp] for temp in line]
          seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
          output = model(seq)

          loss = criterion(output, seq)
          #print(loss.cpu().detach().numpy())
          if (loss.cpu().detach().numpy()>threshold):
            TP += test_abnormal_loader[line]

  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [ ]:
threshold = [0.00001,0.0004,0.0008,0.0001]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  1e-05


Processing Rows: 100%|██████████| 12691/12691 [00:27<00:00, 466.51it/s]


elapsed_time: 163.120s
false positive (FP): 92823, false negative (FN): 0, Precision: 46.609%, Recall: 100.000%, F1-measure: 63.583%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0004


Processing Rows:  10%|▉         | 6259/64372 [00:13<02:05, 463.81it/s]


KeyboardInterrupt: ignored

In [ ]:
threshold = [0.0067,0.0069]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.0067


Processing Rows: 100%|██████████| 3595/3595 [00:45<00:00, 79.64it/s]


elapsed_time: 258.861s
false positive (FP): 15, false negative (FN): 470, Precision: 99.924%, Recall: 97.680%, F1-measure: 98.790%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0069


Processing Rows:   9%|▉         | 1620/17191 [00:20<03:17, 78.82it/s]


KeyboardInterrupt: ignored